In [14]:
from bs4 import BeautifulSoup
import requests

url = 'https://www.pro-football-reference.com'
year = 2018
maxp = 300
    
# grab fantasy players
r = requests.get(url + '/years/' + str(year) + '/fantasy.htm')
soup = BeautifulSoup(r.content, 'html.parser')
parsed_table = soup.find_all('table')[0]  

df = []

# first 2 rows are col headers
for i,row in enumerate(parsed_table.find_all('tr')[2:]):
    if i % 10 == 0: print(i, end=' ')
    if i >= maxp: 
        print('\nComplete.')
        break
        
    try:
        dat = row.find('td', attrs={'data-stat': 'player'})
        name = dat.a.get_text()
        stub = dat.a.get('href')
        stub = stub[:-4] + '/fantasy/' + str(year)
        pos = row.find('td', attrs={'data-stat': 'fantasy_pos'}).get_text()

        # grab this players stats
        tdf = pd.read_html(url + stub)[0]    

        # get rid of MultiIndex, just keep last row
        tdf.columns = tdf.columns.get_level_values(-1)

        # fix the away/home column
        tdf = tdf.rename(columns={'Unnamed: 4_level_2': 'Away'})
        tdf['Away'] = [1 if r=='@' else 0 for r in tdf['Away']]

        # drop all intermediate stats
        tdf = tdf.iloc[:,[1,2,3,4,5,-3]]
        
        # drop "Total" row
        tdf = tdf.query('Date != "Total"')
        
        # add other info
        tdf['Name'] = name
        tdf['Position'] = pos
        tdf['Season'] = year

        df.append(tdf)
    except:
        pass
    
df = pd.concat(df)
df.head()

0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 
Complete.


,G#,Date,Tm,Away,Opp,FantPt,Name,Position,Season
0,1.0,2018-09-10,LAR,1,OAK,20.7,Todd Gurley,RB,2018
1,2.0,2018-09-16,LAR,0,ARI,29.3,Todd Gurley,RB,2018
2,3.0,2018-09-23,LAR,0,LAC,19.6,Todd Gurley,RB,2018
3,4.0,2018-09-27,LAR,0,MIN,21.6,Todd Gurley,RB,2018
4,5.0,2018-10-07,LAR,1,SEA,29.3,Todd Gurley,RB,2018


In [15]:
df.head(100)

,G#,Date,Tm,Away,Opp,FantPt,Name,Position,Season
0,1.0,2018-09-10,LAR,1,OAK,20.7,Todd Gurley,RB,2018
1,2.0,2018-09-16,LAR,0,ARI,29.3,Todd Gurley,RB,2018
2,3.0,2018-09-23,LAR,0,LAC,19.6,Todd Gurley,RB,2018
3,4.0,2018-09-27,LAR,0,MIN,21.6,Todd Gurley,RB,2018
4,5.0,2018-10-07,LAR,1,SEA,29.3,Todd Gurley,RB,2018
...,...,...,...,...,...,...,...,...,...
2,3.0,2018-09-23,DAL,1,SEA,11.8,Ezekiel Elliott,RB,2018
3,4.0,2018-09-30,DAL,0,DET,30.0,Ezekiel Elliott,RB,2018
4,5.0,2018-10-07,DAL,1,HOU,8.4,Ezekiel Elliott,RB,2018
5,6.0,2018-10-14,DAL,0,JAX,17.7,Ezekiel Elliott,RB,2018
